# Pipeline Gold - Analise de Dados e Respostas as Perguntas de Negocio

**MVP - Engenharia de Dados | PUC-Rio**

**Aluno:** Rafael Moreno Soledade Franca  
**Matricula:** 4052025000202

---

## Objetivo deste Notebook

Este notebook representa a **camada final do pipeline de dados** - a camada **Gold**. Aqui transformamos dados limpos e estruturados em **insights acionaveis** que respondem as 13 perguntas de negocio definidas no inicio do projeto.

### O que voce encontrara:

1. **Secao 1:** Modelo de Dados Star Schema utilizado
2. **Secao 2:** Preparacao do ambiente e carregamento dos dados
3. **Secao 3:** Criacao das tabelas Gold (agregacoes)
4. **Secao 4:** Respostas as 13 Perguntas de Negocio
5. **Secao 5:** Conclusoes e Insights Principais

### As 13 Perguntas de Negocio:

| # | Categoria | Pergunta |
|---|-----------|----------|
| 1 | Distribuicao Geografica | Quais regioes concentram a maior densidade populacional? |
| 2 | Distribuicao Geografica | Existe correlacao entre tamanho territorial e populacao? |
| 3 | Distribuicao Geografica | Paises sem acesso ao mar (landlocked) tem caracteristicas populacionais distintas? |
| 4 | Analise Economica | Quais sao as moedas mais utilizadas globalmente? |
| 5 | Analise Economica | Quantos paises compartilham a mesma moeda? |
| 6 | Diversidade Cultural | Quais idiomas tem o maior alcance global? |
| 7 | Diversidade Cultural | Qual a distribuicao de idiomas por regiao geografica? |
| 8 | Rankings Globais | Quais sao os Top 10 paises mais populosos? |
| 9 | Rankings Globais | Quais sao os Top 10 paises com maior area territorial? |
| 10 | Rankings Globais | Quais sao os Top 10 paises com maior densidade populacional? |
| 11 | Metricas Globais | Qual a populacao total mundial? |
| 12 | Metricas Globais | Qual a densidade populacional media global? |
| 13 | Metricas Globais | Quantos paises independentes existem? |

---

# Secao 1: Modelo de Dados - Star Schema

## O que e Star Schema?

O **Star Schema** (Esquema Estrela) e um modelo de dados dimensional amplamente utilizado em Data Warehouses. Ele organiza os dados em:

- **Tabela Fato (centro da estrela):** Contem as metricas numericas mensuraveis
- **Tabelas Dimensao (pontas da estrela):** Contem os atributos descritivos para analise

## Por que Star Schema?

1. **Simplicidade:** Facil de entender e navegar
2. **Performance:** Otimizado para consultas analiticas (poucos JOINs)
3. **Flexibilidade:** Permite responder diversas perguntas de negocio
4. **Padrao de Mercado:** Compativel com ferramentas de BI (Power BI, Tableau)

## Nosso Modelo Star Schema

```
                    +---------------------+
                    |   dim_currencies    |
                    | ------------------- |
                    | currency_code (PK)  |
                    | currency_name       |
                    | currency_symbol     |
                    | country_id (FK)     |
                    +---------+-----------+
                              |
+---------------------+       |       +---------------------+
|    dim_countries    |       |       |    dim_languages    |
| ------------------- |       |       | ------------------- |
| country_id (PK)     |       |       | language_code (PK)  |
| country_name_common |       |       | language_name       |
| region              |       |       | country_id (FK)     |
| subregion           |       v       +----------+----------+
| landlocked          |  +---------------------+ |
| latitude            |  | fact_country_metrics| |
| longitude           |  | ------------------- | |
+---------+-----------+  | country_id (FK/PK)  | |
          |              | population          | |
          +------------->| area_km2            |<+
                         | population_density  |
                         | execution_date      |
                         +---------------------+
```

## Descricao das Tabelas

### Tabela Fato: fact_country_metrics
| Campo | Tipo | Descricao |
|-------|------|-----------|
| country_id | STRING | Chave primaria/estrangeira (codigo ISO) |
| population | LONG | Populacao total do pais |
| area_km2 | DOUBLE | Area territorial em km2 |
| population_density | DOUBLE | Habitantes por km2 |

### Dimensao: dim_countries
| Campo | Tipo | Descricao |
|-------|------|-----------|
| country_id | STRING | Chave primaria (codigo ISO CCA3) |
| country_name_common | STRING | Nome comum do pais |
| region | STRING | Continente |
| subregion | STRING | Sub-regiao geografica |
| landlocked | BOOLEAN | True se nao tem acesso ao mar |

### Dimensao: dim_currencies e dim_languages
Contem os codigos e nomes de moedas e idiomas, vinculados aos paises por country_id.

## Como o Star Schema responde as perguntas de negocio?

| Pergunta | Tabelas Utilizadas | Tipo de Analise |
|----------|-------------------|------------------|
| Densidade por regiao | dim_countries + fact_country_metrics | GROUP BY region |
| Moedas mais usadas | dim_currencies | COUNT + GROUP BY currency_code |
| Top 10 populosos | fact_country_metrics + dim_countries | ORDER BY population DESC |
| Paises landlocked | dim_countries + fact_country_metrics | FILTER landlocked = true |

---

# Secao 2: Preparacao do Ambiente

In [0]:
# Importacoes necessarias
import time
import warnings
from datetime import datetime, date
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

# Suprimir warnings de WindowExpression (esperado para datasets pequenos)
warnings.filterwarnings("ignore", message=".*WindowExpression.*")

# Configuracoes dos schemas e tabelas
SCHEMAS = {
    'bronze': 'workspace.bronze',
    'silver': 'workspace.silver',
    'gold': 'workspace.gold'
}

BRONZE_TABLES = {
    'countries': f"{SCHEMAS['bronze']}.countries_raw"
}

SILVER_TABLES = {
    'dim_countries': f"{SCHEMAS['silver']}.dim_countries",
    'dim_currencies': f"{SCHEMAS['silver']}.dim_currencies",
    'dim_languages': f"{SCHEMAS['silver']}.dim_languages",
    'fact_country_metrics': f"{SCHEMAS['silver']}.fact_country_metrics"
}

GOLD_TABLES = {
    'countries_by_region': f"{SCHEMAS['gold']}.countries_by_region",
    'currency_usage': f"{SCHEMAS['gold']}.currency_usage",
    'language_distribution': f"{SCHEMAS['gold']}.language_distribution",
    'geographic_metrics': f"{SCHEMAS['gold']}.geographic_metrics"
}

def create_database_if_not_exists(spark, schema_name):
    spark.sql(f"CREATE DATABASE IF NOT EXISTS {schema_name}")
    print(f"Database '{schema_name}' verificado/criado")

def optimize_table(spark, table_name):
    spark.sql(f"OPTIMIZE {table_name}")
    print(f"Tabela {table_name} otimizada")

print("Ambiente configurado com sucesso!")
print(f"Data de execucao: {date.today()}")

Ambiente configurado com sucesso!
Data de execucao: 2025-12-09


In [0]:
# Iniciar contagem de tempo
start_time = time.time()

# Criar schema Gold se nao existir
create_database_if_not_exists(spark, SCHEMAS['gold'])

# Carregar tabelas Silver
print("\n" + "="*60)
print("CARREGANDO DADOS DA CAMADA SILVER")
print("="*60)

df_countries = spark.table(SILVER_TABLES['dim_countries'])
df_currencies = spark.table(SILVER_TABLES['dim_currencies'])
df_languages = spark.table(SILVER_TABLES['dim_languages'])
df_metrics = spark.table(SILVER_TABLES['fact_country_metrics'])

print(f"\ndim_countries: {df_countries.count()} registros")
print(f"dim_currencies: {df_currencies.count()} registros")
print(f"dim_languages: {df_languages.count()} registros")
print(f"fact_country_metrics: {df_metrics.count()} registros")

Database 'workspace.gold' verificado/criado

CARREGANDO DADOS DA CAMADA SILVER

dim_countries: 195 registros
dim_currencies: 146 registros
dim_languages: 140 registros
fact_country_metrics: 195 registros


In [0]:
# Criar view consolidada (JOIN entre dimensao countries e fato metrics)
# IMPORTANTE: Deduplicar para evitar contagens infladas
df_metrics_unique = df_metrics.dropDuplicates(['country_id'])

df_country_full = df_countries.join(
    df_metrics_unique,
    'country_id',
    'inner'
)

print(f"\nView consolidada criada: {df_country_full.count()} paises com metricas")


View consolidada criada: 195 paises com metricas


---

# Secao 3: Criacao das Tabelas Gold

As tabelas Gold sao agregacoes pre-calculadas que otimizam consultas analiticas e alimentam dashboards.

In [0]:
# =====================================================
# TABELA GOLD 1: countries_by_region
# Estatisticas agregadas por regiao e sub-regiao
# =====================================================

print("\n" + "="*60)
print("CRIANDO: countries_by_region")
print("="*60)

df_countries_by_region = df_country_full.groupBy('region', 'subregion').agg(
    count('*').alias('total_countries'),
    sum('population').alias('total_population'),
    sum('area_km2').alias('total_area_km2'),
    round(avg('population'), 0).alias('avg_population'),
    round(avg('area_km2'), 2).alias('avg_area_km2'),
    round(avg('population_density'), 2).alias('avg_population_density'),
    sum(when(col('landlocked') == True, 1).otherwise(0)).alias('landlocked_count'),
    max('population').alias('max_population'),
    min('population').alias('min_population')
).orderBy(col('total_population').desc())

df_countries_by_region = df_countries_by_region.withColumn('calculation_date', lit(date.today()))

# Salvar
df_countries_by_region.write.format('delta').partitionBy('calculation_date').mode('append').saveAsTable(GOLD_TABLES['countries_by_region'])

print(f"\nTabela salva: {GOLD_TABLES['countries_by_region']}")
print(f"Registros: {df_countries_by_region.count()}")


CRIANDO: countries_by_region

Tabela salva: workspace.gold.countries_by_region
Registros: 24


In [0]:
# =====================================================
# TABELA GOLD 2: currency_usage
# Ranking de moedas por numero de paises
# =====================================================

print("\n" + "="*60)
print("CRIANDO: currency_usage")
print("="*60)

# Ler Bronze para re-explodir currencies
df_bronze_latest = spark.table(BRONZE_TABLES['countries']) \
    .filter(col('execution_date') == spark.table(BRONZE_TABLES['countries']).agg(max('execution_date')).collect()[0][0])

country_schema = StructType([
    StructField('cca3', StringType()),
    StructField('name', StructType([StructField('common', StringType())])),
    StructField('currencies', MapType(StringType(), StructType([
        StructField('name', StringType()),
        StructField('symbol', StringType())
    ])))
])

df_countries_curr = df_bronze_latest.withColumn('parsed', from_json(col('data'), country_schema))

df_currency_usage = df_countries_curr.select(
    col('parsed.cca3').alias('country_id'),
    col('parsed.name.common').alias('country_name'),
    explode(col('parsed.currencies')).alias('currency_code', 'currency_data')
).select(
    'currency_code',
    col('currency_data.name').alias('currency_name'),
    'country_name'
).groupBy('currency_code', 'currency_name').agg(
    collect_set('country_name').alias('countries_using'),
    countDistinct('country_name').alias('total_countries')
).orderBy(col('total_countries').desc())

# Salvar
df_currency_usage.write.format('delta').mode('overwrite').saveAsTable(GOLD_TABLES['currency_usage'])

print(f"\nTabela salva: {GOLD_TABLES['currency_usage']}")
print(f"Registros: {df_currency_usage.count()}")


CRIANDO: currency_usage

Tabela salva: workspace.gold.currency_usage
Registros: 146


In [0]:
# =====================================================
# TABELA GOLD 3: language_distribution
# Ranking de idiomas por alcance global
# =====================================================

print("\n" + "="*60)
print("CRIANDO: language_distribution")
print("="*60)

lang_schema = StructType([
    StructField('cca3', StringType()),
    StructField('name', StructType([StructField('common', StringType())])),
    StructField('languages', MapType(StringType(), StringType()))
])

df_countries_lang = df_bronze_latest.withColumn('parsed', from_json(col('data'), lang_schema))

df_language_distribution = df_countries_lang.select(
    col('parsed.name.common').alias('country_name'),
    explode(col('parsed.languages')).alias('language_code', 'language_name')
).groupBy('language_code', 'language_name').agg(
    collect_set('country_name').alias('countries_speaking'),
    countDistinct('country_name').alias('total_countries')
).orderBy(col('total_countries').desc())

# Salvar
df_language_distribution.write.format('delta').mode('overwrite').saveAsTable(GOLD_TABLES['language_distribution'])

print(f"\nTabela salva: {GOLD_TABLES['language_distribution']}")
print(f"Registros: {df_language_distribution.count()}")


CRIANDO: language_distribution

Tabela salva: workspace.gold.language_distribution
Registros: 141


In [0]:
# =====================================================
# TABELA GOLD 4: geographic_metrics
# KPIs globais consolidados
# =====================================================

print("\n" + "="*60)
print("CRIANDO: geographic_metrics")
print("="*60)

calc_date = date.today()

# Calcular metricas globais
total_countries = df_country_full.count()
world_population = df_country_full.agg(sum('population')).collect()[0][0]
world_area = df_country_full.agg(sum('area_km2')).collect()[0][0]
avg_country_density = df_country_full.agg(avg('population_density')).collect()[0][0]
world_density = world_population / world_area
landlocked_total = df_country_full.filter(col('landlocked') == True).count()
total_regions = df_country_full.select('region').distinct().count()
largest_country_area = df_country_full.agg(max('area_km2')).collect()[0][0]
largest_country_pop = df_country_full.agg(max('population')).collect()[0][0]

metrics_data = [
    ('total_countries', float(total_countries), calc_date),
    ('world_population', float(world_population), calc_date),
    ('world_land_area_km2', float(world_area), calc_date),
    ('avg_country_density', float(avg_country_density), calc_date),
    ('world_population_density', float(world_density), calc_date),
    ('landlocked_countries', float(landlocked_total), calc_date),
    ('total_regions', float(total_regions), calc_date),
    ('largest_country_area_km2', float(largest_country_area), calc_date),
    ('largest_country_population', float(largest_country_pop), calc_date)
]

df_geographic_metrics = spark.createDataFrame(metrics_data, ['metric_name', 'metric_value', 'calculation_date'])

# Salvar
df_geographic_metrics.write.format('delta').partitionBy('calculation_date').mode('append').saveAsTable(GOLD_TABLES['geographic_metrics'])

print(f"\nTabela salva: {GOLD_TABLES['geographic_metrics']}")
print(f"Registros: {df_geographic_metrics.count()}")


CRIANDO: geographic_metrics

Tabela salva: workspace.gold.geographic_metrics
Registros: 9


In [0]:
# =====================================================
# DOCUMENTAÇÃO UNITY CATALOG - CAMADA GOLD
# =====================================================
# Registrar metadados de todas as 4 tabelas Gold (tabelas + colunas)

print("\n" + "="*60)
print("REGISTRANDO DOCUMENTAÇÃO NO UNITY CATALOG")
print("="*60)

# =====================================================
# 1. countries_by_region - Propriedades e Colunas
# =====================================================
spark.sql("""
ALTER TABLE workspace.gold.countries_by_region 
SET TBLPROPERTIES (
    'comment' = 'Agregacao de metricas por regiao e sub-regiao. Responde perguntas sobre distribuicao geografica e densidade populacional. 24 sub-regioes.',
    'model' = 'Aggregation Table',
    'layer' = 'gold',
    'grain' = '1 registro por sub-regiao',
    'source_tables' = 'silver.dim_countries, silver.fact_country_metrics'
)
""")

countries_by_region_columns = {
    'region': 'Nome do continente (Africa, Americas, Asia, Europe, Oceania)',
    'subregion': 'Nome da sub-regiao geografica dentro do continente',
    'total_countries': 'Quantidade de paises na sub-regiao',
    'total_population': 'Soma da populacao de todos os paises da sub-regiao',
    'total_area_km2': 'Soma da area territorial em km2 de todos os paises',
    'avg_population': 'Media de populacao por pais na sub-regiao',
    'avg_area_km2': 'Media de area territorial por pais na sub-regiao',
    'avg_population_density': 'Media da densidade populacional (hab/km2) na sub-regiao',
    'landlocked_count': 'Quantidade de paises sem acesso ao mar na sub-regiao',
    'max_population': 'Maior populacao entre os paises da sub-regiao',
    'min_population': 'Menor populacao entre os paises da sub-regiao',
    'calculation_date': 'Data de calculo das metricas (particao)'
}

for column_name, comment in countries_by_region_columns.items():
    spark.sql(f"ALTER TABLE workspace.gold.countries_by_region ALTER COLUMN {column_name} COMMENT '{comment}'")

print("✓ countries_by_region documentada (tabela + 12 colunas)")

# =====================================================
# 2. currency_usage - Propriedades e Colunas
# =====================================================
spark.sql("""
ALTER TABLE workspace.gold.currency_usage 
SET TBLPROPERTIES (
    'comment' = 'Ranking de moedas por numero de paises que as utilizam e lista de paises. Responde perguntas de analise economica. 146 moedas.',
    'model' = 'Aggregation Table',
    'layer' = 'gold',
    'grain' = '1 registro por moeda',
    'source_tables' = 'bronze.countries_raw'
)
""")

currency_usage_columns = {
    'currency_code': 'Codigo ISO da moeda (ex: EUR, USD, BRL)',
    'currency_name': 'Nome completo da moeda (ex: Euro, United States dollar)',
    'countries_using': 'Lista de paises que utilizam esta moeda como oficial',
    'total_countries': 'Quantidade de paises que utilizam esta moeda'
}

for column_name, comment in currency_usage_columns.items():
    spark.sql(f"ALTER TABLE workspace.gold.currency_usage ALTER COLUMN {column_name} COMMENT '{comment}'")

print("✓ currency_usage documentada (tabela + 4 colunas)")

# =====================================================
# 3. language_distribution - Propriedades e Colunas
# =====================================================
spark.sql("""
ALTER TABLE workspace.gold.language_distribution 
SET TBLPROPERTIES (
    'comment' = 'Ranking de idiomas por alcance global (numero de paises onde sao oficiais). Responde perguntas de diversidade cultural. 141 idiomas.',
    'model' = 'Aggregation Table',
    'layer' = 'gold',
    'grain' = '1 registro por idioma',
    'source_tables' = 'bronze.countries_raw'
)
""")

language_distribution_columns = {
    'language_code': 'Codigo ISO 639-3 do idioma (ex: eng, fra, por)',
    'language_name': 'Nome do idioma em ingles (ex: English, French, Portuguese)',
    'countries_speaking': 'Lista de paises onde o idioma e oficial ou co-oficial',
    'total_countries': 'Quantidade de paises onde o idioma e oficial'
}

for column_name, comment in language_distribution_columns.items():
    spark.sql(f"ALTER TABLE workspace.gold.language_distribution ALTER COLUMN {column_name} COMMENT '{comment}'")

print("✓ language_distribution documentada (tabela + 4 colunas)")

# =====================================================
# 4. geographic_metrics - Propriedades e Colunas
# =====================================================
spark.sql("""
ALTER TABLE workspace.gold.geographic_metrics 
SET TBLPROPERTIES (
    'comment' = 'KPIs globais consolidados: populacao mundial, area total, densidade media, contagem de paises. Metricas executivas. 9 KPIs.',
    'model' = 'KPI Table',
    'layer' = 'gold',
    'grain' = 'Metricas globais unicas por data',
    'source_tables' = 'silver.dim_countries, silver.fact_country_metrics'
)
""")

geographic_metrics_columns = {
    'metric_name': 'Nome do KPI (ex: total_countries, world_population, world_land_area_km2)',
    'metric_value': 'Valor numerico do KPI',
    'calculation_date': 'Data de calculo do KPI (particao)'
}

for column_name, comment in geographic_metrics_columns.items():
    spark.sql(f"ALTER TABLE workspace.gold.geographic_metrics ALTER COLUMN {column_name} COMMENT '{comment}'")

print("✓ geographic_metrics documentada (tabela + 3 colunas)")

print("\n" + "="*60)
print("✓ Documentação Unity Catalog completa!")
print("  - 4 tabelas com TBLPROPERTIES")
print("  - 23 colunas documentadas com COMMENT")
print("="*60)



REGISTRANDO DOCUMENTAÇÃO NO UNITY CATALOG
✓ countries_by_region documentada (tabela + 12 colunas)
✓ currency_usage documentada (tabela + 4 colunas)
✓ language_distribution documentada (tabela + 4 colunas)
✓ geographic_metrics documentada (tabela + 3 colunas)

✓ Documentação Unity Catalog completa!
  - 4 tabelas com TBLPROPERTIES
  - 23 colunas documentadas com COMMENT


---

# Secao 4: Respostas as Perguntas de Negocio

Agora vamos responder cada uma das 13 perguntas de negocio definidas no objetivo do MVP, utilizando os dados processados nas camadas Silver e Gold.

---

## Categoria 1: Distribuicao Populacional e Geografica

### Pergunta 1: Quais regioes concentram a maior densidade populacional?

**Contexto:** A densidade populacional indica quantas pessoas vivem por quilometro quadrado. Regioes com alta densidade enfrentam desafios de infraestrutura, habitacao e recursos naturais, enquanto regioes de baixa densidade podem ter dificuldades de conexao e logistica.

In [0]:
print("="*70)
print("PERGUNTA 1: Quais regioes concentram a maior densidade populacional?")
print("="*70)

df_density_by_region = df_country_full.groupBy('region', 'subregion').agg(
    round(avg('population_density'), 2).alias('densidade_media_hab_km2'),
    sum('population').alias('populacao_total'),
    sum('area_km2').alias('area_total_km2'),
    count('*').alias('num_paises')
).orderBy(col('densidade_media_hab_km2').desc())

print("\nRESULTADO: Densidade Populacional Media por Regiao/Sub-regiao")
print("(Ordenado por densidade - maior para menor)\n")

df_density_by_region.show(15, truncate=False)

PERGUNTA 1: Quais regioes concentram a maior densidade populacional?

RESULTADO: Densidade Populacional Media por Regiao/Sub-regiao
(Ordenado por densidade - maior para menor)

+--------+------------------+-----------------------+---------------+--------------+----------+
|region  |subregion         |densidade_media_hab_km2|populacao_total|area_total_km2|num_paises|
+--------+------------------+-----------------------+---------------+--------------+----------+
|Europe  |Western Europe    |2617.02                |189613339      |1017447.02    |8         |
|Asia    |South-Eastern Asia|909.47                 |684495959      |4537819.0     |11        |
|Europe  |Southern Europe   |556.69                 |131534221      |1041504.49    |9         |
|Asia    |Southern Asia     |475.25                 |2011599527     |6782838.0     |9         |
|Asia    |Western Asia      |288.35                 |296352968      |4815760.0     |16        |
|Americas|Caribbean         |256.48                 |37

**Discussao do Resultado:**

- **Western Europe** lidera em densidade media (2.617 hab/km2), impulsionada por Monaco, Malta e paises pequenos muito densos
- **South-Eastern Asia** vem em segundo (909 hab/km2), influenciada por Singapura (8.606 hab/km2)
- **Southern Asia** concentra a maior populacao total (India, Bangladesh, Paquistao) com densidade de 475 hab/km2
- **Oceania** e partes da **Americas** apresentam as menores densidades

**Insight de Negocio:** Empresas que buscam grande volume de consumidores em espacos concentrados devem focar na Europa Ocidental e Asia. Para operacoes que requerem espaco fisico (logistica, agricultura), Americas e Oceania sao mais adequadas.

---

### Pergunta 2: Existe correlacao entre tamanho territorial e populacao?

**Contexto:** Intuitivamente, poderiamos pensar que paises maiores tem mais habitantes. Mas sera que essa relacao e verdadeira? Fatores como clima, geografia e desenvolvimento historico podem influenciar.

In [0]:
print("="*70)
print("PERGUNTA 2: Existe correlacao entre tamanho territorial e populacao?")
print("="*70)

# Comparar Top 10 por area vs Top 10 por populacao
print("\nTOP 10 PAISES POR AREA TERRITORIAL:")
df_country_full.select(
    'country_name_common',
    round(col('area_km2')/1000000, 2).alias('area_milhoes_km2'),
    round(col('population')/1000000, 1).alias('populacao_milhoes'),
    round('population_density', 1).alias('densidade')
).orderBy(col('area_km2').desc()).show(10, truncate=False)

print("\nTOP 10 PAISES POR POPULACAO:")
df_country_full.select(
    'country_name_common',
    round(col('population')/1000000, 1).alias('populacao_milhoes'),
    round(col('area_km2')/1000000, 2).alias('area_milhoes_km2'),
    round('population_density', 1).alias('densidade')
).orderBy(col('population').desc()).show(10, truncate=False)

# Calcular coeficiente de correlacao
correlation = df_country_full.stat.corr('area_km2', 'population')
print(f"\nCOEFICIENTE DE CORRELACAO (Area vs Populacao): {correlation:.4f}")

PERGUNTA 2: Existe correlacao entre tamanho territorial e populacao?

TOP 10 PAISES POR AREA TERRITORIAL:
+-------------------+----------------+-----------------+---------+
|country_name_common|area_milhoes_km2|populacao_milhoes|densidade|
+-------------------+----------------+-----------------+---------+
|Russia             |17.1            |146.0            |8.5      |
|Canada             |9.98            |41.7             |4.2      |
|China              |9.71            |1408.3           |145.1    |
|United States      |9.53            |340.1            |35.7     |
|Brazil             |8.52            |213.4            |25.1     |
|Australia          |7.69            |27.5             |3.6      |
|India              |3.29            |1417.5           |431.2    |
|Argentina          |2.78            |46.7             |16.8     |
|Kazakhstan         |2.72            |20.4             |7.5      |
|Algeria            |2.38            |47.4             |19.9     |
+-------------------+--

**Discussao do Resultado:**

- O coeficiente de correlacao e **positivo mas moderado** (~0.37), indicando que area e populacao tem alguma relacao, mas nao e determinante
- **Russia** e o maior pais em area (17M km2), mas tem apenas 146M habitantes
- **India** tem 3.3M km2 mas lidera em populacao com 1.4 bilhoes
- **Bangladesh** e um exemplo extremo: area pequena (148K km2) com 170M de pessoas

**Insight de Negocio:** O tamanho territorial **nao e um bom preditor** de populacao ou mercado consumidor. Fatores como clima, acesso a agua, desenvolvimento economico e historia sao mais relevantes.

---

### Pergunta 3: Paises sem acesso ao mar (landlocked) tem caracteristicas populacionais distintas?

**Contexto:** Paises landlocked dependem de vizinhos para comercio maritimo, o que historicamente limitou seu desenvolvimento. Sera que isso se reflete em suas caracteristicas demograficas?

In [0]:
print("="*70)
print("PERGUNTA 3: Paises landlocked tem caracteristicas distintas?")
print("="*70)

# Comparar landlocked vs paises com costa
df_landlocked_comparison = df_country_full.groupBy('landlocked').agg(
    count('*').alias('quantidade_paises'),
    round(avg('population'), 0).alias('populacao_media'),
    round(avg('area_km2'), 0).alias('area_media_km2'),
    round(avg('population_density'), 2).alias('densidade_media'),
    sum('population').alias('populacao_total')
)

print("\nCOMPARACAO: Paises Landlocked vs Com Acesso ao Mar")
df_landlocked_comparison.show(truncate=False)

# Listar paises landlocked
landlocked_countries = df_country_full.filter(col('landlocked') == True).select(
    'country_name_common', 'region', 
    round(col('population')/1000000, 1).alias('pop_milhoes'),
    round('population_density', 1).alias('densidade')
).orderBy(col('population').desc())

print(f"\nLISTA DE PAISES LANDLOCKED ({landlocked_countries.count()} paises):")
landlocked_countries.show(20, truncate=False)

PERGUNTA 3: Paises landlocked tem caracteristicas distintas?

COMPARACAO: Paises Landlocked vs Com Acesso ao Mar
+----------+-----------------+---------------+--------------+---------------+---------------+
|landlocked|quantidade_paises|populacao_media|area_media_km2|densidade_media|populacao_total|
+----------+-----------------+---------------+--------------+---------------+---------------+
|false     |150              |4.8975981E7    |773338.0      |359.67         |7346397177     |
|true      |45               |1.3956314E7    |390385.0      |151.13         |628034136      |
+----------+-----------------+---------------+--------------+---------------+---------------+


LISTA DE PAISES LANDLOCKED (45 paises):
+-------------------+--------+-----------+---------+
|country_name_common|region  |pop_milhoes|densidade|
+-------------------+--------+-----------+---------+
|Ethiopia           |Africa  |111.7      |101.1    |
|Uganda             |Africa  |45.9       |190.0    |
|Afghanistan    

**Discussao do Resultado:**

- Existem **45 paises landlocked** (23% do total de 195)
- A **populacao media** e significativamente menor que paises com costa (14M vs 49M)
- A **densidade media** dos paises landlocked e bem menor (151 hab/km2 vs 360 hab/km2)
- **Etiopia** e o pais landlocked mais populoso (111M), seguido por **Uganda** (46M) e **Afeganistao** (44M)

**Insight de Negocio:** Paises landlocked tendem a ter economias menores, populacoes menores e comercio internacional mais limitado. Para empresas de logistica, esses paises representam desafios adicionais de distribuicao.

---

## Categoria 2: Analise Economica e Monetaria

### Pergunta 4: Quais sao as moedas mais utilizadas globalmente?

**Contexto:** O sistema monetario global e fragmentado em centenas de moedas, mas algumas dominam o comercio internacional. Identificar as moedas mais usadas ajuda em decisoes de expansao de negocios e gestao cambial.

In [0]:
print("="*70)
print("PERGUNTA 4: Quais sao as moedas mais utilizadas globalmente?")
print("="*70)

df_top_currencies = spark.table(GOLD_TABLES['currency_usage']).select(
    'currency_code',
    'currency_name',
    'total_countries',
    'countries_using'
).orderBy(col('total_countries').desc())

total_currencies = df_top_currencies.count()
print(f"\nTotal de moedas catalogadas: {total_currencies}")
print("\nTOP 15 MOEDAS MAIS UTILIZADAS (por numero de paises):")

df_top_currencies.select(
    'currency_code', 'currency_name', 'total_countries'
).show(15, truncate=False)

PERGUNTA 4: Quais sao as moedas mais utilizadas globalmente?

Total de moedas catalogadas: 146

TOP 15 MOEDAS MAIS UTILIZADAS (por numero de paises):
+-------------+-------------------------+---------------+
|currency_code|currency_name            |total_countries|
+-------------+-------------------------+---------------+
|EUR          |Euro                     |26             |
|USD          |United States dollar     |10             |
|XOF          |West African CFA franc   |8              |
|XAF          |Central African CFA franc|6              |
|XCD          |Eastern Caribbean dollar |6              |
|AUD          |Australian dollar        |4              |
|ZAR          |South African rand       |4              |
|INR          |Indian rupee             |2              |
|SGD          |Singapore dollar         |2              |
|CHF          |Swiss franc              |2              |
|WST          |Samoan tālā              |1              |
|DZD          |Algerian dinar         

**Discussao do Resultado:**

- O **Euro (EUR)** lidera com 26 paises que o utilizam (zona do euro + paises que o adotam)
- O **Dolar Americano (USD)** e usado em 10 paises (EUA + economias dolarizadas)
- O **Franco CFA** (XOF com 8 paises + XAF com 6) une a Africa Ocidental e Central
- O **Dolar do Caribe Oriental (XCD)** e usado por 6 paises caribenhos
- Existem **146 moedas unicas** catalogadas no mundo

**Insight de Negocio:** Uma plataforma de e-commerce que suporte EUR, USD e as principais moedas regionais pode atender a grande maioria dos paises do mundo. O Euro e especialmente relevante por cobrir a maior economia integrada.

---

### Pergunta 5: Quantos paises compartilham a mesma moeda?

**Contexto:** Zonas monetarias (como a Zona do Euro) facilitam o comercio entre membros, eliminando custos de conversao cambial. Entender essas zonas e crucial para estrategias de expansao regional.

In [0]:
print("="*70)
print("PERGUNTA 5: Quantos paises compartilham a mesma moeda?")
print("="*70)

df_shared_currencies = spark.table(GOLD_TABLES['currency_usage']).filter(
    col('total_countries') > 1
).select(
    'currency_code',
    'currency_name',
    'total_countries',
    'countries_using'
).orderBy(col('total_countries').desc())

print(f"\nMoedas compartilhadas por multiplos paises: {df_shared_currencies.count()}")
print("\nPRINCIPAIS ZONAS MONETARIAS:")

df_shared_currencies.show(10, truncate=50)

PERGUNTA 5: Quantos paises compartilham a mesma moeda?

Moedas compartilhadas por multiplos paises: 10

PRINCIPAIS ZONAS MONETARIAS:
+-------------+-------------------------+---------------+--------------------------------------------------+
|currency_code|            currency_name|total_countries|                                   countries_using|
+-------------+-------------------------+---------------+--------------------------------------------------+
|          EUR|                     Euro|             26|[Greece, Portugal, Spain, Slovenia, Finland, Ne...|
|          USD|     United States dollar|             10|[Cambodia, Timor-Leste, El Salvador, Ecuador, P...|
|          XOF|   West African CFA franc|              8|[Niger, Mali, Benin, Guinea-Bissau, Burkina Fas...|
|          XAF|Central African CFA franc|              6|[Gabon, Chad, Equatorial Guinea, Central Africa...|
|          XCD| Eastern Caribbean dollar|              6|[Saint Vincent and the Grenadines, Saint Kitts 

**Discussao do Resultado:**

- **Zona do Euro (EUR):** Maior bloco monetario multi-nacional com 26 paises utilizando
- **Dolar Americano (USD):** Usado em 10 paises, incluindo economias dolarizadas (Equador, El Salvador, Panama)
- **Franco CFA:** Heranca colonial francesa - XOF (8 paises) e XAF (6 paises) na Africa
- **Dolar do Caribe Oriental (XCD):** Une 6 pequenas nacoes caribenhas

**Insight de Negocio:** Zonas monetarias compartilhadas reduzem barreiras de entrada para negocios. Empresas podem escalar operacoes dentro da zona sem custos cambiais adicionais.

---

## Categoria 3: Diversidade Cultural e Linguistica

### Pergunta 6: Quais idiomas tem o maior alcance global?

**Contexto:** O idioma e uma das maiores barreiras para expansao internacional de produtos e servicos. Identificar idiomas de maior alcance ajuda a priorizar esforcos de localizacao.

In [0]:
print("="*70)
print("PERGUNTA 6: Quais idiomas tem o maior alcance global?")
print("="*70)

df_top_languages = spark.table(GOLD_TABLES['language_distribution']).select(
    'language_code',
    'language_name',
    'total_countries',
    'countries_speaking'
).orderBy(col('total_countries').desc())

total_languages = df_top_languages.count()
print(f"\nTotal de idiomas catalogados: {total_languages}")
print("\nTOP 20 IDIOMAS COM MAIOR ALCANCE GLOBAL:")

df_top_languages.select(
    'language_code', 'language_name', 'total_countries'
).show(20, truncate=False)

PERGUNTA 6: Quais idiomas tem o maior alcance global?

Total de idiomas catalogados: 141

TOP 20 IDIOMAS COM MAIOR ALCANCE GLOBAL:
+-------------+-------------+---------------+
|language_code|language_name|total_countries|
+-------------+-------------+---------------+
|eng          |English      |59             |
|fra          |French       |31             |
|ara          |Arabic       |24             |
|spa          |Spanish      |21             |
|por          |Portuguese   |9              |
|rus          |Russian      |7              |
|deu          |German       |6              |
|swa          |Swahili      |4              |
|tsn          |Tswana       |4              |
|ita          |Italian      |4              |
|msa          |Malay        |3              |
|tam          |Tamil        |3              |
|grn          |Guaraní      |3              |
|srp          |Serbian      |3              |
|nld          |Dutch        |3              |
|cat          |Catalan      |3           

**Discussao do Resultado:**

- **Ingles (eng):** Idioma oficial ou co-oficial em 59 paises (heranca do Imperio Britanico)
- **Frances (fra):** Segundo idioma mais presente com 31 paises, especialmente na Africa
- **Arabe (ara):** Dominante no Oriente Medio e Norte da Africa com 24 paises
- **Espanhol (spa):** Quarto mais presente com 21 paises, concentrado nas Americas
- **Portugues (por):** Presente em 9 paises (Brasil + CPLP + outras regioes)

**Insight de Negocio:** Uma empresa que localize produtos para **Ingles, Frances, Arabe, Espanhol e Portugues** pode atender a grande maioria dos paises do mundo, cobrindo praticamente todas as regioes.

---

### Pergunta 7: Qual a distribuicao de idiomas por regiao geografica?

**Contexto:** A distribuicao linguistica reflete padroes historicos de colonizacao e migracao. Entender isso ajuda a definir estrategias regionais de comunicacao.

In [0]:
print("="*70)
print("PERGUNTA 7: Distribuicao de idiomas por regiao geografica")
print("="*70)

# Re-explodir languages com regiao
full_schema = StructType([
    StructField('name', StructType([StructField('common', StringType())])),
    StructField('region', StringType()),
    StructField('languages', MapType(StringType(), StringType()))
])

df_lang_region = df_bronze_latest.withColumn('parsed', from_json(col('data'), full_schema))

df_lang_by_region = df_lang_region.select(
    col('parsed.region').alias('region'),
    explode(col('parsed.languages')).alias('lang_code', 'lang_name')
).groupBy('region', 'lang_name').agg(
    count('*').alias('num_paises')
).orderBy('region', col('num_paises').desc())

# Top idiomas por regiao
window = Window.partitionBy('region').orderBy(col('num_paises').desc())
df_top_lang_region = df_lang_by_region.withColumn('rank', row_number().over(window)).filter(col('rank') <= 5)

print("\nTOP 5 IDIOMAS POR REGIAO:")
df_top_lang_region.select('region', 'lang_name', 'num_paises').show(30, truncate=False)

PERGUNTA 7: Distribuicao de idiomas por regiao geografica

TOP 5 IDIOMAS POR REGIAO:
+--------+-------------------------+----------+
|region  |lang_name                |num_paises|
+--------+-------------------------+----------+
|Africa  |English                  |69        |
|Africa  |French                   |66        |
|Africa  |Arabic                   |36        |
|Africa  |Portuguese               |18        |
|Africa  |Tswana                   |12        |
|Americas|Spanish                  |57        |
|Americas|English                  |42        |
|Americas|Guaraní                  |9         |
|Americas|Quechua                  |6         |
|Americas|Aymara                   |6         |
|Asia    |Arabic                   |36        |
|Asia    |English                  |15        |
|Asia    |Russian                  |15        |
|Asia    |Malay                    |9         |
|Asia    |Tamil                    |9         |
|Europe  |German                   |15        |
|Eu

**Discussao do Resultado:**

- **Africa:** Ingles (69 paises), Frances (66) e Arabe (36) dominam - heranca colonial
- **Americas:** Espanhol (57 paises) e Ingles (42) sao predominantes; linguas indigenas como Guarani e Quechua tambem presentes
- **Asia:** Arabe (36 paises), Ingles e Russo (15 cada); grande diversidade de linguas locais
- **Europa:** Alemao (15), Frances (15), Italiano (12) - multiplicidade de idiomas nacionais
- **Oceania:** Ingles predominante (42 paises) - influencia britanica

**Insight de Negocio:** Estrategias de localizacao devem ser regionais. Para Africa, priorizar Ingles e Frances. Para Americas, Espanhol e Ingles. Para Asia, considerar Arabe e linguas locais.

---

## Categoria 4: Rankings e Comparacoes Globais

### Pergunta 8: Quais sao os Top 10 paises mais populosos?

In [0]:
print("="*70)
print("PERGUNTA 8: Top 10 paises mais populosos")
print("="*70)

print("\nTOP 10 PAISES POR POPULACAO:")
df_country_full.select(
    row_number().over(Window.orderBy(col('population').desc())).alias('Rank'),
    col('country_name_common').alias('Pais'),
    col('region').alias('Regiao'),
    format_number(col('population'), 0).alias('Populacao'),
    format_number(col('area_km2'), 0).alias('Area_km2'),
    round(col('population_density'), 1).alias('Densidade')
).show(10, truncate=False)

PERGUNTA 8: Top 10 paises mais populosos

TOP 10 PAISES POR POPULACAO:
+----+-------------+--------+-------------+----------+---------+
|Rank|Pais         |Regiao  |Populacao    |Area_km2  |Densidade|
+----+-------------+--------+-------------+----------+---------+
|1   |India        |Asia    |1,417,492,000|3,287,263 |431.2    |
|2   |China        |Asia    |1,408,280,000|9,706,961 |145.1    |
|3   |United States|Americas|340,110,988  |9,525,067 |35.7     |
|4   |Indonesia    |Asia    |284,438,782  |1,904,569 |149.3    |
|5   |Pakistan     |Asia    |241,499,431  |796,095   |303.4    |
|6   |Nigeria      |Africa  |223,800,000  |923,768   |242.3    |
|7   |Brazil       |Americas|213,421,037  |8,515,767 |25.1     |
|8   |Bangladesh   |Asia    |169,828,911  |147,570   |1150.8   |
|9   |Russia       |Europe  |146,028,325  |17,098,246|8.5      |
|10  |Mexico       |Americas|130,575,786  |1,964,375 |66.5     |
+----+-------------+--------+-------------+----------+---------+
only showing top 10

**Discussao:** Os Top 10 paises mais populosos concentram aproximadamente **58% da populacao mundial**. India recentemente ultrapassou a China como pais mais populoso. A Asia domina com 6 paises no ranking.

### Pergunta 9: Quais sao os Top 10 paises com maior area territorial?

In [0]:
print("="*70)
print("PERGUNTA 9: Top 10 paises por area territorial")
print("="*70)

print("\nTOP 10 PAISES POR AREA:")
df_country_full.select(
    row_number().over(Window.orderBy(col('area_km2').desc())).alias('Rank'),
    col('country_name_common').alias('Pais'),
    col('region').alias('Regiao'),
    format_number(col('area_km2'), 0).alias('Area_km2'),
    format_number(col('population'), 0).alias('Populacao'),
    round(col('population_density'), 1).alias('Densidade')
).show(10, truncate=False)

PERGUNTA 9: Top 10 paises por area territorial

TOP 10 PAISES POR AREA:
+----+-------------+--------+----------+-------------+---------+
|Rank|Pais         |Regiao  |Area_km2  |Populacao    |Densidade|
+----+-------------+--------+----------+-------------+---------+
|1   |Russia       |Europe  |17,098,246|146,028,325  |8.5      |
|2   |Canada       |Americas|9,984,670 |41,651,653   |4.2      |
|3   |China        |Asia    |9,706,961 |1,408,280,000|145.1    |
|4   |United States|Americas|9,525,067 |340,110,988  |35.7     |
|5   |Brazil       |Americas|8,515,767 |213,421,037  |25.1     |
|6   |Australia    |Oceania |7,692,024 |27,536,874   |3.6      |
|7   |India        |Asia    |3,287,263 |1,417,492,000|431.2    |
|8   |Argentina    |Americas|2,780,400 |46,735,004   |16.8     |
|9   |Kazakhstan   |Asia    |2,724,900 |20,426,568   |7.5      |
|10  |Algeria      |Africa  |2,381,741 |47,400,000   |19.9     |
+----+-------------+--------+----------+-------------+---------+
only showing top 1

**Discussao:** A Russia sozinha representa **11% da area terrestre do planeta**. Os Top 10 paises por area cobrem mais de **50% da superficie terrestre**. Note como paises extensos como Russia, Canada e Australia tem baixa densidade populacional.

### Pergunta 10: Quais sao os Top 10 paises com maior densidade populacional?

In [0]:
print("="*70)
print("PERGUNTA 10: Top 10 paises por densidade populacional")
print("="*70)

print("\nTOP 10 PAISES POR DENSIDADE POPULACIONAL (hab/km2):")
df_country_full.filter(col('area_km2') > 0).select(
    row_number().over(Window.orderBy(col('population_density').desc())).alias('Rank'),
    col('country_name_common').alias('Pais'),
    col('region').alias('Regiao'),
    round(col('population_density'), 1).alias('Densidade_hab_km2'),
    format_number(col('population'), 0).alias('Populacao'),
    format_number(col('area_km2'), 0).alias('Area_km2')
).show(10, truncate=False)

PERGUNTA 10: Top 10 paises por densidade populacional

TOP 10 PAISES POR DENSIDADE POPULACIONAL (hab/km2):
+----+------------+--------+-----------------+-----------+--------+
|Rank|Pais        |Regiao  |Densidade_hab_km2|Populacao  |Area_km2|
+----+------------+--------+-----------------+-----------+--------+
|1   |Monaco      |Europe  |19021.3          |38,423     |2       |
|2   |Singapore   |Asia    |8605.9           |6,110,200  |710     |
|3   |Bahrain     |Asia    |2084.5           |1,594,654  |765     |
|4   |Malta       |Europe  |1817.2           |574,250    |316     |
|5   |Vatican City|Europe  |1800.0           |882        |0       |
|6   |Maldives    |Asia    |1717.1           |515,132    |300     |
|7   |Bangladesh  |Asia    |1150.8           |169,828,911|147,570 |
|8   |Barbados    |Americas|622.8            |267,800    |430     |
|9   |Mauritius   |Africa  |609.7            |1,243,741  |2,040   |
|10  |San Marino  |Europe  |559.5            |34,132     |61      |
+----+---

**Discussao:** Monaco lidera com impressionantes **19.000+ hab/km2**, mas e uma cidade-estado de apenas 2 km2. Bangladesh e o caso mais notavel: um pais de tamanho consideravel com densidade extrema (1.150 hab/km2), enfrentando serios desafios de urbanizacao.

---

## Categoria 5: Metricas Globais e KPIs

### Pergunta 11: Qual a populacao total mundial?

In [0]:
print("="*70)
print("PERGUNTA 11: Qual a populacao total mundial?")
print("="*70)

world_pop = df_country_full.agg(sum('population').alias('total')).collect()[0]['total']
world_pop_billions = world_pop / 1_000_000_000

print(f"\nPOPULACAO MUNDIAL TOTAL: {world_pop:,.0f} habitantes")
print(f"Equivalente a: {world_pop_billions:.2f} bilhoes de pessoas")

# Distribuicao por regiao
print("\nDISTRIBUICAO POR REGIAO:")
df_pop_by_region = df_country_full.groupBy('region').agg(
    sum('population').alias('populacao'),
    count('*').alias('num_paises')
).withColumn(
    'percentual', round(col('populacao') / world_pop * 100, 1)
).orderBy(col('populacao').desc())

df_pop_by_region.select(
    'region',
    format_number('populacao', 0).alias('Populacao'),
    col('percentual').alias('Percentual_do_Total'),
    'num_paises'
).show(truncate=False)

PERGUNTA 11: Qual a populacao total mundial?

POPULACAO MUNDIAL TOTAL: 7,974,431,313 habitantes
Equivalente a: 7.97 bilhoes de pessoas

DISTRIBUICAO POR REGIAO:
+--------+-------------+-------------------+----------+
|region  |Populacao    |Percentual_do_Total|num_paises|
+--------+-------------+-------------------+----------+
|Asia    |4,687,718,085|58.8               |46        |
|Africa  |1,460,640,780|18.3               |54        |
|Americas|1,037,563,923|13.0               |35        |
|Europe  |741,279,275  |9.3                |46        |
|Oceania |47,229,250   |0.6                |14        |
+--------+-------------+-------------------+----------+



**Discussao:** A populacao mundial esta em aproximadamente **7.97 bilhoes** de pessoas. A Asia sozinha concentra **59% da populacao mundial**, demonstrando a importancia demografica e economica do continente.

### Pergunta 12: Qual a densidade populacional media global?

In [0]:
print("="*70)
print("PERGUNTA 12: Qual a densidade populacional media global?")
print("="*70)

# Duas formas de calcular densidade
world_area = df_country_full.agg(sum('area_km2')).collect()[0][0]
world_pop = df_country_full.agg(sum('population')).collect()[0][0]
avg_country_density = df_country_full.agg(avg('population_density')).collect()[0][0]
world_real_density = world_pop / world_area

print(f"\nMETRICAS DE DENSIDADE:")
print(f"")
print(f"1. Media aritmetica das densidades dos paises: {avg_country_density:.2f} hab/km2")
print(f"   (Cada pais tem peso igual, independente do tamanho)")
print(f"")
print(f"2. Densidade populacional global real: {world_real_density:.2f} hab/km2")
print(f"   (Populacao total / Area total)")
print(f"")
print(f"Por que a diferenca? Paises pequenos e muito densos (Monaco, Singapura)")
print(f"elevam a media aritmetica, mas tem pouco impacto na densidade global real.")

PERGUNTA 12: Qual a densidade populacional media global?

METRICAS DE DENSIDADE:

1. Media aritmetica das densidades dos paises: 311.54 hab/km2
   (Cada pais tem peso igual, independente do tamanho)

2. Densidade populacional global real: 59.70 hab/km2
   (Populacao total / Area total)

Por que a diferenca? Paises pequenos e muito densos (Monaco, Singapura)
elevam a media aritmetica, mas tem pouco impacto na densidade global real.


**Discussao:** Ha duas formas de calcular "densidade media":

1. **Media das densidades (~311 hab/km2):** Cada pais tem peso igual. Monaco (19.000) eleva muito essa media.
2. **Densidade global real (~60 hab/km2):** Populacao total dividida por area total. Reflete melhor a distribuicao real.

A segunda metrica e mais util para analises de recursos globais.

### Pergunta 13: Quantos paises independentes existem?

In [0]:
print("="*70)
print("PERGUNTA 13: Quantos paises independentes existem?")
print("="*70)

total_countries = df_country_full.count()
total_regions = df_country_full.select('region').distinct().count()
total_subregions = df_country_full.select('subregion').distinct().count()
landlocked = df_country_full.filter(col('landlocked') == True).count()

print(f"\nRESUMO GLOBAL:")
print(f"")
print(f"Paises independentes: {total_countries}")
print(f"Regioes (continentes): {total_regions}")
print(f"Sub-regioes: {total_subregions}")
print(f"Paises landlocked: {landlocked} ({landlocked/total_countries*100:.1f}%)")
print(f"Paises com costa: {total_countries - landlocked} ({(total_countries-landlocked)/total_countries*100:.1f}%)")

# Distribuicao por regiao
print(f"\nPAISES POR REGIAO:")
df_country_full.groupBy('region').agg(
    count('*').alias('Total_de_Paises')
).orderBy(col('Total_de_Paises').desc()).show(truncate=False)

PERGUNTA 13: Quantos paises independentes existem?

RESUMO GLOBAL:

Paises independentes: 195
Regioes (continentes): 5
Sub-regioes: 24
Paises landlocked: 45 (23.1%)
Paises com costa: 150 (76.9%)

PAISES POR REGIAO:
+--------+---------------+
|region  |Total_de_Paises|
+--------+---------------+
|Africa  |54             |
|Asia    |46             |
|Europe  |46             |
|Americas|35             |
|Oceania |14             |
+--------+---------------+



**Discussao:** Existem **195 paises independentes** reconhecidos internacionalmente. A Africa e o continente com mais paises (54), resultado da fragmentacao pos-colonial. A Oceania tem menos paises, mas muitos sao pequenas nacoes insulares.

---

# Secao 5: Conclusoes e Insights Principais

In [0]:
# Otimizar tabelas Gold
print("\nOtimizando tabelas Gold...")
for table in GOLD_TABLES.values():
    try:
        optimize_table(spark, table)
    except:
        pass

# Tempo de execucao
execution_time = time.time() - start_time

print("\n" + "="*70)
print("RESUMO FINAL DA EXECUCAO")
print("="*70)
print(f"\nTabelas Gold criadas: 4")
print(f"  - countries_by_region")
print(f"  - currency_usage")
print(f"  - language_distribution")
print(f"  - geographic_metrics")
print(f"\nPerguntas de negocio respondidas: 13/13 (100%)")
print(f"Tempo de execucao: {execution_time:.2f} segundos")
print(f"Status: SUCESSO")
print(f"Data de execucao: {datetime.now()}")
print("="*70)


Otimizando tabelas Gold...
Tabela workspace.gold.countries_by_region otimizada
Tabela workspace.gold.currency_usage otimizada
Tabela workspace.gold.language_distribution otimizada
Tabela workspace.gold.geographic_metrics otimizada

RESUMO FINAL DA EXECUCAO

Tabelas Gold criadas: 4
  - countries_by_region
  - currency_usage
  - language_distribution
  - geographic_metrics

Perguntas de negocio respondidas: 13/13 (100%)
Tempo de execucao: 141.55 segundos
Status: SUCESSO
Data de execucao: 2025-12-09 16:49:12.408143


## Principais Insights Descobertos

### 1. Distribuicao Populacional
- **59% da populacao mundial** esta concentrada na Asia
- Os **Top 10 paises mais populosos** representam 58% da humanidade
- **Nao ha correlacao forte** entre area territorial e populacao

### 2. Divisao Geografica
- **45 paises (23%)** sao landlocked, enfrentando desafios de comercio internacional
- **Russia sozinha** representa 11% da area terrestre do planeta
- Densidade varia de **2 hab/km2 (Mongolia)** a **19.000 hab/km2 (Monaco)**

### 3. Economia e Moedas
- O **Euro (EUR)** e a moeda mais utilizada (26 paises)
- O **Dolar Americano (USD)** e usado em 10 paises
- **146 moedas unicas** catalogadas no mundo
- Zonas monetarias (Euro, Franco CFA, Dolar) facilitam comercio regional

### 4. Diversidade Linguistica
- **Ingles** e o idioma oficial/co-oficial em 59 paises
- **Frances** em 31 paises, **Arabe** em 24, **Espanhol** em 21
- Localizar para **5 idiomas** (Ingles, Frances, Arabe, Espanhol, Portugues) cobre a maioria dos paises
- Distribuicao linguistica reflete historia colonial

---

## Valor para Negocios

| Area | Insight Acionavel |
|------|-------------------|
| E-commerce | Priorizar EUR, USD e principais moedas regionais |
| Localizacao | Investir primeiro em Ingles, Frances, Espanhol |
| Expansao | Asia concentra maior mercado consumidor potencial |
| Logistica | Paises landlocked requerem estrategias especiais |
| Marketing | Adaptar mensagens para densidade urbana de cada regiao |

---

## Pipeline Completo: Bronze -> Silver -> Gold

```
+-------------------------------------------------------------+
| BRONZE: Dados Brutos                                        |
| - 2 APIs publicas (REST Countries, Exchange Rate)           |
| - 195 paises em formato JSON                                |
| - Preservacao historica com timestamps                      |
+-------------------------------------------------------------+
                            |
                            v
+-------------------------------------------------------------+
| SILVER: Dados Estruturados (Star Schema)                    |
| - 1 Tabela Fato: fact_country_metrics                       |
| - 3 Dimensoes: dim_countries, dim_currencies, dim_languages |
| - Validacoes de qualidade e integridade                     |
+-------------------------------------------------------------+
                            |
                            v
+-------------------------------------------------------------+
| GOLD: Insights Acionaveis                                   |
| - 4 Tabelas analiticas pre-agregadas                        |
| - 13 Perguntas de negocio respondidas                       |
| - Rankings, KPIs e metricas globais                         |
| - Pronto para dashboards e ferramentas de BI                |
+-------------------------------------------------------------+
```